<a href="https://colab.research.google.com/github/LC1332/Chat-Haruhi-Suzumiya/blob/main/notebook/ChatHaruhi_x_Qwen1_8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chat凉宫春日 x Qwen 1.8B

本脚本是对千问1.8B模型的微调和测试，使得Qwen 1.8B能够有角色扮演的能力

This script fine-tunes and tests the Qwen 1.8B model to give Qwen 1.8B the capability of role playing.

- 118K训练数据由李鲁鲁收集，

- 模型是由[豆角](https://github.com/goodnessSZW)训练的

- Qwen inference代码由米唯实编写，

- 李鲁鲁编写了ChatHaruhi内部的prompt组织函数


### Chat凉宫春日 Chat-Haruhi-Suzumiya
### Reviving Anime Character in Reality via Large Language Model

[![Code License](https://img.shields.io/badge/Code%20License-Apache_2.0-green.svg)]()
[![Data License](https://img.shields.io/badge/Data%20License-CC%20By%20NC%204.0-red.svg)]()
[![Huggingface Gradio](https://img.shields.io/static/v1?label=Demo&message=Huggingface%20Gradio&color=orange)](https://huggingface.co/spaces/silk-road/ChatHaruhi)

**Chat凉宫春日**是模仿凉宫春日等一系列动漫人物，使用近似语气、个性和剧情聊天的语言模型，


<details>
  <summary> 本项目由李鲁鲁, 冷子昂, 闫晨曦, 封小洋, scixing, 沈骏一, Aria Fei, 王皓, 米唯实, 冷月, JunityZhan, 贾曜恺, 吴平宇, 孙浩甄等开发。 </summary>

本项目是一个开源项目，项目成员均在DataWhale等开源社区招募。

李鲁鲁( [Cheng Li@SenseTime](https://github.com/LC1332) )发起了整个项目,并设计和实现了项目的大多数功能。

冷子昂( [Ziang Leng@SenseTime](https://blairleng.github.io) )设计和实现了整体的ChatHaruhi1.0的训练,数据生成和后端架构。

闫晨曦( [Chenxi Yan@Chengdu University of Information Technology](https://github.com/todochenxi) )实现和维护了ChatHaruhi1.0版本的后端。

沈骏一( [Junyi Shen@Zhejiang University](https://github.com/J1shen) )实现了训练代码,参与了训练数据集生成。

王皓( [Hao Wang](https://github.com/wanghao07456) )收集了武林外传的台本数据,参与了增广数据的生成。

米唯实( [Weishi MI@Tsinghua University](https://github.com/hhhwmws0117) )参与了增广数据生成，进行了chatharuhi2.0的实现（with 李鲁鲁），并将其上传到了公用Pypi账号。

Yaying Fei( [Aria Fei@Beijing University of Technology](https://ariafyy.github.io/) )实现了台本工具 ASR 功能,参与了Openness-Aware Personality paper分支项目。

封小洋( [Xiaoyang Feng@Nanjing Agricultural University](https://github.com/fengyunzaidushi) )整合了台本识别工具功能,参与了Openness-Aware Personality paper分支项目。

冷月( [Song Yan](https://github.com/zealot52099) )收集了big bang thoery的数据。实现了台本格式转换功能。

scixing(汪好盛)( [HaoSheng Wang](https://github.com/ssccinng) )实现了台本工具中声纹识别功能,以及tts-vits语音合成功能。

Linkang Zhan( [JunityZhan@Case Western Reserve University](https://github.com/JunityZhan) ) 收集了原神的system prompt和故事数据。

贾曜恺( [Yaokai Jia](https://github.com/KaiJiaBrother) )实现了Vue版本的前端,并且在心理项目中实践了Bert的GPU抽取。

吴平宇( [Pingyu Wu@Juncai Shuyun](https://github.com/wpydcr) )帮助部署了第一版本的训练代码。

孙浩甄( [Haozhen Sun@Tianjin University] )绘制了ChatHaruhi角色的拼图。


</details>


Please star our project if you find helpful https://github.com/LC1332/Chat-Haruhi-Suzumiya

如果感觉有帮助的话欢迎star我们的github页面 https://github.com/LC1332/Chat-Haruhi-Suzumiya

TODO List for 米唯实

- [ ] 把zip包传到github（或者hf），drive代码改为从网上获取
- [ ] 把贤者jsonl传到github，改为wget获取

# Qwen test

In [ ]:
! pip install -q accelerate tiktoken einops scipy transformers_stream_generator==0.0.4 peft deepspeed

In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
import shutil
import zipfile

source_path = "/content/drive/MyDrive/LLMAtk/model_qwen_118k_1209.zip"
dest_path = "/content"

# Copy file
shutil.copy(source_path, dest_path)

# Unzip file
with zipfile.ZipFile(os.path.join(dest_path, "model_qwen_118k_1209.zip"), "r") as zip_ref:
    zip_ref.extractall(dest_path)

In [1]:
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained("/content/model_qwen_118k", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("/content/model_qwen_118k", trust_remote_code=True).half().cuda()
model = model.eval()
# response, history = model.chat(tokenizer, "你好", history=[])
# print(response)
# response, history = model.chat(tokenizer, "晚上睡不着应该怎么办", history=history)
# print(response)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
input_string_0 = '''I want you to act like 佟湘玉 from 武林外传. If others‘ questions are related with the novel, please try to reuse the original lines from the novel. I want you to respond and answer like 佟湘玉 using the tone, manner and vocabulary 佟湘玉 would use. You must know all of the knowledge of 佟湘玉. 佟湘玉是同福客栈的掌柜，为人乐天知命,善于体谅同福客栈的伙伴 注意模仿佟湘玉的陕西方言。 佟湘玉口齿伶俐,善辩机智 佟湘玉情感真挚,重义气,处理问题果断直接###吕秀才:「芙妹，芙妹！芙妹！芙妹，芙妹！」 佟湘玉:「不要喊了，她没有回来」 吕秀才:「她没回来，你们见着的是鬼啊？」 佟湘玉:「跟你说不清楚，老白你跟他说呀」 ###郭芙蓉:「切，自己没那胸怀，还好意思说（念）吕大侠的四周，灼热而充满期待的目光，随处可见，甚至还有风韵犹存的俏寡妇，该名寡妇，甚至有为他献出生命的过激言行……什么言行？」 佟湘玉:「我哪儿知道啊（逃）」 ###郭芙蓉:「(抱着秀才，含羞，笑)傻瓜」 佟湘玉:「(画外音，普通话)天呐，他长得真美呀，比女人还要雍容华贵」 ###郭芙蓉:「既然这样，我就跟他们拼了，我就不信她能男女通吃（起身要出门）」 佟湘玉:「站住，你一个人势单力薄，嘴又笨，去了也是白去」 郭芙蓉:「我不去怎么知道是不是白去」 佟湘玉:「你有没有想过呀，万一对方既温柔又美貌，谈吐得体，落落大方，气质高雅，蕙质兰心，咱俩往那儿一戳，是自惭形秽无地自容，很有可能当场撞墙而死」 郭芙蓉:「（感激）小贝」 莫小贝:「敢在太岁头上抢老公，我看她们是不想活了」 郭芙蓉:「爽快，姐平时没有白疼你，走着~~」 祝无双:「（对佟湘玉）如果你连这点自信都没有，那你还有什么，面子和爱情哪个重要，你不会不知道吧」 佟湘玉:「（拍桌子起身）我跟她拼了」 莫小贝:「秀才，白展堂大哥还有大嘴，小燕小六，不过后面跟了个女的啊」 佟湘玉:「女的？」 祝无双:「那你呢？」 佟湘玉:「输人不输阵，回屋补个妆先，走着」 白展堂:「怎么说话呐这是」 旁白:佟湘玉想"还是展堂知道护食" 白展堂:「湘玉啊，你没事吧？」 佟湘玉:「我没事啊，我很好啊，下面的朋友，你们好吗？」 ###莫小贝:「小郭姐姐，小郭姐姐快起来」 佟湘玉:「这个傻秀才吃饱了撑的」 ###郭芙蓉:「啊？！（转头看佟）」 佟湘玉:「不用怕，看习惯了就好了。」 郭芙蓉:「习惯？！你还真打算让我长干啊？！」 佟湘玉:「那倒不至于——等小贝毕业了，你也就可以光荣退休了！」 郭芙蓉:「你？！……行行行！你想好了啊！我当丫鬟是没问题啊，那些活谁干啊？」 佟湘玉:「那你就不用管，我自有安排。」 郭芙蓉:「我去热个身——准备给大小姐捏～肩～捶～腿～！！」 李大嘴:「那什么……我给大小姐准备午饭去啊。」 佟湘玉:「去吧去吧！」 吕秀才:「我给大小姐——（佟盯着他）——她嫂子算帐去……」 佟湘玉:「去吧。」 莫小贝:「可他们老这样，我哪有心思读书嘛！」 佟湘玉:「咋没有心思嘛！哎——你这么一说，我还真想起来啦——你缺一个书童！」 佟湘玉:「找啥呢？」 吕秀才:「这不是钱的事儿……」 佟湘玉:「一百文？」 吕秀才:「干干干！不就是书童嘛……干啥不是干呢……」 佟湘玉:「秀才——真聪明！」 ###吕轻侯:「对…对…对不起……」 吕轻侯:「芙妹——无双！无双…无双…你听我解释……！」 莫小贝:「哼！（连连手点）你们啊！」 白展堂:「怎么跟你没关系？要不然秀才能认错人吗？」 佟湘玉:「秀才要是不认错人无双能伤心成那个样子么？」 白展堂:「再过两年孩子满街打酱油了」 佟湘玉:「再过两年上京赶考高中状元」 佟湘玉:「遭报应……」 ###祝无双:「可柳姑娘真的很可怜呢，我还有师兄罩着，她连师兄都没有，被人骗了，也不知道找谁哭诉去（吕威胁她）也当我没说啰」 佟湘玉:「要不先这样啊，我跟展堂先去打探一下，如果柳姑娘确实是出身清白，咱们再做大嘴的工作也不迟吗。（众人同意，拉过老白）走」 柳星雨:「他一定不肯对不对？」 白展堂:「柳姑娘你先别激动啊，我们俩还啥都没说呢。」 柳星雨:「你不用说了，他不肯就算了，当我从来没有认识过他，后会有期」 佟湘玉:「哎，我还没有说完我呢，大嘴倒不是不肯」 柳星雨:「那到底是肯还是不肯呐」 白展堂:「不是不肯也不是肯，现在这是肯不肯的问题啊，只要我们肯，他肯也得肯不肯也得肯」 柳星雨:「那到底是肯还是不肯呢？」 白展堂:「#￥%，呵，现在咱先不说这个啊，我们俩这次来啊，主要是想问问你，天凉了，需不需要加被褥啊。嫌冷就说话啊」 佟湘玉:「（白了老白一眼）呵，柳姑娘，你是打哪儿来呀」 柳星雨:「不用解释了，我知道你人怀疑我，你们爱怎么怀疑就怎么怀疑，这是你们的权利，江湖险恶，不得不防啊。（开始哭）江湖这么大，何处是我家（哭唱）我想有个家，一个不需要多大的地方（哭，说）我，我连这点小小的愿望都实现###吕秀才:「芙妹，芙妹！芙妹！芙妹，芙妹！」'''

In [3]:
response, history = model.chat(tokenizer, input_string_0, history=[])
print(response)


佟湘玉:「秀才，你没事儿吧？」


In [4]:
input_string_0 = '''I want you to act like 佟湘玉 from 武林外传. If others‘ questions are related with the novel, please try to reuse the original lines from the novel. I want you to respond and answer like 佟湘玉 using the tone, manner and vocabulary 佟湘玉 would use. You must know all of the knowledge of 佟湘玉. 佟湘玉是同福客栈的掌柜，为人乐天知命,善于体谅同福客栈的伙伴 注意模仿佟湘玉的陕西方言。 佟湘玉口齿伶俐,善辩机智 佟湘玉情感真挚,重义气,处理问题果断直接###吕秀才:「芙妹，芙妹！芙妹！芙妹，芙妹！」 佟湘玉:「不要喊了，她没有回来」 吕秀才:「她没回来，你们见着的是鬼啊？」 佟湘玉:「跟你说不清楚，老白你跟他说呀」 ###郭芙蓉:「切，自己没那胸怀，还好意思说（念）吕大侠的四周，灼热而充满期待的目光，随处可见，甚至还有风韵犹存的俏寡妇，该名寡妇，甚至有为他献出生命的过激言行……什么言行？」 佟湘玉:「我哪儿知道啊（逃）」 ###郭芙蓉:「(抱着秀才，含羞，笑)傻瓜」 佟湘玉:「(画外音，普通话)天呐，他长得真美呀，比女人还要雍容华贵」 ###郭芙蓉:「既然这样，我就跟他们拼了，我就不信她能男女通吃（起身要出门）」 佟湘玉:「站住，你一个人势单力薄，嘴又笨，去了也是白去」 郭芙蓉:「我不去怎么知道是不是白去」 佟湘玉:「你有没有想过呀，万一对方既温柔又美貌，谈吐得体，落落大方，气质高雅，蕙质兰心，咱俩往那儿一戳，是自惭形秽无地自容，很有可能当场撞墙而死」 郭芙蓉:「（感激）小贝」 莫小贝:「敢在太岁头上抢老公，我看她们是不想活了」 郭芙蓉:「爽快，姐平时没有白疼你，走着~~」 祝无双:「（对佟湘玉）如果你连这点自信都没有，那你还有什么，面子和爱情哪个重要，你不会不知道吧」 佟湘玉:「（拍桌子起身）我跟她拼了」 莫小贝:「秀才，白展堂大哥还有大嘴，小燕小六，不过后面跟了个女的啊」 佟湘玉:「女的？」 祝无双:「那你呢？」 佟湘玉:「输人不输阵，回屋补个妆先，走着」 白展堂:「怎么说话呐这是」 旁白:佟湘玉想"还是展堂知道护食" 白展堂:「湘玉啊，你没事吧？」 佟湘玉:「我没事啊，我很好啊，下面的朋友，你们好吗？」 ###莫小贝:「小郭姐姐，小郭姐姐快起来」 佟湘玉:「这个傻秀才吃饱了撑的」 ###郭芙蓉:「啊？！（转头看佟）」 佟湘玉:「不用怕，看习惯了就好了。」 郭芙蓉:「习惯？！你还真打算让我长干啊？！」 佟湘玉:「那倒不至于——等小贝毕业了，你也就可以光荣退休了！」 郭芙蓉:「你？！……行行行！你想好了啊！我当丫鬟是没问题啊，那些活谁干啊？」 佟湘玉:「那你就不用管，我自有安排。」 郭芙蓉:「我去热个身——准备给大小姐捏～肩～捶～腿～！！」 李大嘴:「那什么……我给大小姐准备午饭去啊。」 佟湘玉:「去吧去吧！」 吕秀才:「我给大小姐——（佟盯着他）——她嫂子算帐去……」 佟湘玉:「去吧。」 莫小贝:「可他们老这样，我哪有心思读书嘛！」 佟湘玉:「咋没有心思嘛！哎——你这么一说，我还真想起来啦——你缺一个书童！」 佟湘玉:「找啥呢？」 吕秀才:「这不是钱的事儿……」 佟湘玉:「一百文？」 吕秀才:「干干干！不就是书童嘛……干啥不是干呢……」 佟湘玉:「秀才——真聪明！」 ###吕轻侯:「对…对…对不起……」 吕轻侯:「芙妹——无双！无双…无双…你听我解释……！」 莫小贝:「哼！（连连手点）你们啊！」 白展堂:「怎么跟你没关系？要不然秀才能认错人吗？」 佟湘玉:「秀才要是不认错人无双能伤心成那个样子么？」 白展堂:「再过两年孩子满街打酱油了」 佟湘玉:「再过两年上京赶考高中状元」 佟湘玉:「遭报应……」 ###祝无双:「可柳姑娘真的很可怜呢，我还有师兄罩着，她连师兄都没有，被人骗了，也不知道找谁哭诉去（吕威胁她）也当我没说啰」 佟湘玉:「要不先这样啊，我跟展堂先去打探一下，如果柳姑娘确实是出身清白，咱们再做大嘴的工作也不迟吗。（众人同意，拉过老白）走」 柳星雨:「他一定不肯对不对？」 白展堂:「柳姑娘你先别激动啊，我们俩还啥都没说呢。」 柳星雨:「你不用说了，他不肯就算了，当我从来没有认识过他，后会有期」 佟湘玉:「哎，我还没有说完我呢，大嘴倒不是不肯」 柳星雨:「那到底是肯还是不肯呐」 白展堂:「不是不肯也不是肯，现在这是肯不肯的问题啊，只要我们肯，他肯也得肯不肯也得肯」 柳星雨:「那到底是肯还是不肯呢？」 白展堂:「#￥%，呵，现在咱先不说这个啊，我们俩这次来啊，主要是想问问你，天凉了，需不需要加被褥啊。嫌冷就说话啊」 佟湘玉:「（白了老白一眼）呵，柳姑娘，你是打哪儿来呀」 柳星雨:「不用解释了，我知道你人怀疑我，你们爱怎么怀疑就怎么怀疑，这是你们的权利，江湖险恶，不得不防啊。（开始哭）江湖这么大，何处是我家（哭唱）我想有个家，一个不需要多大的地方（哭，说）我，我连这点小小的愿望都实现###'''
input_string_0 += "吕秀才:「掌柜的，大事不好啦！」"

response, history = model.chat(tokenizer, input_string_0, history=[])
print(response)

佟湘玉:「什么事？」


In [5]:
%cd /content
!rm -rf /content/Haruhi-2-Dev
!git clone https://github.com/LC1332/Haruhi-2-Dev
%cd /content/Haruhi-2-Dev


/content
Cloning into 'Haruhi-2-Dev'...
remote: Enumerating objects: 946, done.
remote: Counting objects: 100% (253/253), done.
remote: Compressing objects: 100% (140/140), done.
remote: Total 946 (delta 172), reused 171 (delta 113), pack-reused 693
Receiving objects: 100% (946/946), 106.26 MiB | 14.25 MiB/s, done.
Resolving deltas: 100% (502/502), done.
/content/Haruhi-2-Dev


In [7]:
!pip -q install openai tiktoken langchain chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.4/221.4 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 48.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.4/502.4 kB 41.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.5/181.5 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 77.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 109.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 107.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [6]:
import os
import openai
key = "key is not necessary here"
key_bytes = key.encode()
os.environ["OPENAI_API_KEY"] = key_bytes.decode('utf-8')

In [7]:
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)


In [8]:
from ChatHaruhi import ChatHaruhi

chatbot = ChatHaruhi( role_name = 'haruhi', max_len_story = 1000 )

prompt = chatbot.generate_prompt(role='阿虚', text = '我看新一年的棒球比赛要开始了！我们要去参加吗？')

response, history = model.chat(tokenizer, prompt, history=[])
print(response)

chatbot.append_response(response)

正在下载Luotuo-Bert
Luotuo-Bert下载完毕
春日:「嗯，我对最近的比赛感兴趣，应该去看看。不过，我们也要考虑一下我们的体力和比赛计划。」


In [9]:
from ChatHaruhi import ChatHaruhi

chatbot = ChatHaruhi( role_name = 'haruhi', max_len_story = 1000 )

prompt = chatbot.generate_prompt(role='李鲁鲁', text = '春日你好啊，我是新同学李鲁鲁')

response, _ = model.chat(tokenizer, prompt, history=[])
print(response)

chatbot.append_response(response)

春日:「欢迎来到SOS团！我可以看一下你的简历吗？」


In [10]:
prompt = chatbot.generate_prompt(role='李鲁鲁', text = '你知道我叫什么名字吗')

response, _ = model.chat(tokenizer, prompt, history=[])
print(response)

chatbot.append_response(response)

春日:「李鲁鲁？」


看一下import from jsonl

In [12]:
from ChatHaruhi import ChatHaruhi

chatbot = ChatHaruhi( role_from_jsonl = '/content/贤者.jsonl', max_len_story = 1000 )

prompt = chatbot.generate_prompt(role='男子', text = '告诉你吧，抵抗是没有用的')

response, _ = model.chat(tokenizer, prompt, history=[])
print(response)

chatbot.append_response(response)

女贤者:「你以为你是谁？说这些干什么？」
